# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # WRITE YOUR CODE HERE
           cols <- c("a", "x", "c")
      sim_dat = data.frame(matrix(nrow = sample_size, ncol = length(cols))) 
      colnames(sim_dat) = cols
      sim_dat[,1] <- runif(sample_size, age_lo, age_hi)

      for (i in 1:sample_size) {
        noise_x <- rnorm(1, 0, sd_x)
        noise_c <- rnorm(1, 0, sd_c)
        sim_dat[i,2] <- (beta_xa * sim_dat[i,1]) + beta_x0 + noise_x
        sim_dat[i,3] <- (beta_ca * sim_dat[i,1]) + (beta_cx * sim_dat[i,2]) + beta_c0 + noise_c
      }
      return (sim_dat)
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,a,x,c
,<dbl>,<dbl>,<dbl>
1,186.18390,104.08812,650.1076
2,181.91310,64.58186,345.7605
3,172.51197,152.84554,619.0402
4,164.18918,78.95552,412.1988
5,86.15816,181.41483,677.4196
6,197.20379,92.55168,409.6035


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [5]:
# WRITE YOUR CODE HERE
install.packages("mediation")
library(mediation)
run_analysis <- function(sim_dat) {
  fitM <- lm(x ~ a, data=sim_dat)
  fitY <- lm(c ~ x + a, data=sim_dat)
  fitMed <- mediate(fitM, fitY, treat="a", mediator="x")
  return (c(fitMed$d0, fitMed$d0.p, fitMed$z0, fitMed$z0.p))
}

med_values <- run_analysis(dat)
print(med_values)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0




[1] 1.8992391 0.0000000 0.3390733 0.2620000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [17]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(dat)
    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov = mean(simouts[,4] <= alpha)
    
    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [18]:
# WRITE YOUR CODE HERE
cov <- repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
print(cov)

$ACME_cov
[1] 1

$ADE_cov
[1] 0.7



---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [27]:
# WRITE YOUR CODE HERE
sample_sizes <- c(50, 75, 100, 125, 150)
cov_vals <- data.frame(matrix(nrow = length(sample_sizes), ncol = 3))
colnames(cov_vals) = c("sample_size", "ACME_cov", "ADE_cov")
cov_vals[,1] = sample_sizes

for (i in 1:5) {
  cov <- repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size = sample_sizes[i], age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
  cov_vals[i,2:3] <- cov
}


Print your results. 

In [28]:
# WRITE YOUR CODE HERE
print(cov_vals)

  sample_size ACME_cov ADE_cov
1          50      0.4     0.2
2          75      0.7     0.7
3         100      0.9     0.6
4         125      0.9     0.8
5         150      1.0     1.0


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> Based on my results, I would conclude that a sample size of 150 is needed in order to correctly reject the null 100% of the time (at 0.01 type I error rate) for both the direct and indirect effects in the mediation analysis.
> 

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> The direct effect was harder to detect that the indirect effect because reading experience values were calculated based on the generated age values, then we generated our reading comprehension values and took into account both reading experience and age when doing so. Therefore, we calculated our reading comprehension values around the stucture of the indirect pathway, building in the influence of age on reading experience (the mediating variable).

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*